# FIT5196-S1-2019 Assessment 1  
## Task - 1
### Student Name: Mranali Mehta
### Student ID: 29778271
Date: 14/04/2018
Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

##### Libraries used :
re : for regular expression for extracting the data.

json : for creating json document.


### Explanation of the task-1
The task is about extracting the unit information from a text file which is basically a scrapped web-page containing html tags.

<li>The task specify to extract the unit_id, unit_title, synopsis, requirements, pre-requisites, co-requisites, outcomes and chief examiner of the unit and store this information in two different formats: json and xml.</li>
<li>To accomplish the task, the text file is read and saved as a list of tokens in a variable</li>
<li>The loop is used to iterate over the list. Some flag variables are used for the puurspose of error handling and keep count of certain lines.</li>
<li>While the loop iterate over the list of token, one by one the each of the information is extracted using the regular expression.</li>
<li>While the unit information are extracted we also need to store them in a proper format,so during extracting the information from the text file's tokens following things are performed in parallel :</li>
<ol>The information is extracted from the text.</ol>
<ol>Once the information is in variable, it is written to an xml file with proper tags.</ol>
<ol>also it is stored n the dictionary at the same time which can be used later to dump into a json file.</ol>

The process used is pretty straight forward.

In [8]:
# Importing the required andallowed libraries
import re
import json

# Open the input text file to read the content
text_file = open("29778271.txt", 'r')
textfile_tokens = text_file.readlines()
line = 0
unit = 0

# Open the output xml file to write into it & initialize the file with initial tags and count the line number.
# I have done counting of line number because of an error occuring at certain tag. In order to handle that error it was required.
xml_output = open("29778271.xml",'w')
xml_output.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
line += 1
xml_output.write("<units>\n")
line += 1

# Dictionary to hold extracted data are initialised for forming the JSON file later in the code.
main_dictionary = {}
unit_dictionary = {}
unit_details = []

#Unit code & title : The regular expression used for extracting the unit code
unitcode_regex = r"(<h1) (class=\"[a-z]+_[a-z]+\"><span) (class=\"unitcode\">)([A-Z]{3}[0-9]{4})(</span>)"

#Chief Examiner : The regex used for extracting chief examiner
ch_ex_regex = r"<p class=\"hbk-highlight-heading\">Chief examiner\(s\)</p>"
chex_name_regex = r"(<a href=\"http\://staffsearch\.monash\.edu/\?name=)(.*)(\">)"
coordinator_regex = r"<p class=\"hbk-highlight-heading\">Coordinator\(s\)</p>"

# Assessment/Requirements : The regex used for extracting assessment or requirements.
assess_regex = r"<h2 class=\"hbk-heading\">Assessment</h2>"
require_ptag_regex = r"(<p>)(.*)(</p>)"
require_litag_regex = r"(<li>)(.*)(</li>)"
workload_regex = r"<h2 class=\"hbk-heading\">Workload requirements</h2>"
last_tag_regex = r"<p class=\"visuallyhidden\" id=\"breadcrumb__label\">You are here:</p>"

# Synopsis and Outcomes : regex for synopsis and outcomes 
synop_regex = r"<h2 class=\"hbk-heading\">Synopsis</h2>"
synop_ptag_regex = r"(<p>)(.*)(</p>)"
synop_litag_regex = r"(<li>)(.*)(</li>)"
outcome_regex = r"<h2 class=\"hbk-heading\">Outcomes</h2>"

# Prerequisites & Corequisites : Regex for pre-requisites and co-requisites.
prereq_regex = r"<p class=\"hbk-preamble-heading\">Prerequisites</p>"
pre_regex = r"[A-Z]{3}[0-9]{4}"
prohibition_regex = r"<p class=\"hbk-preamble-heading\">Prohibitions</p>"

for index,token in enumerate(textfile_tokens):
    
    #Saving one unit details and initialising for the first time
    if (re.search(r"<div class=\"hbk-banner-box\">", token)):
        if unit == 0:
            xml_output.write("\t<unit>\n")
            line += 1
            unit_dict = {}
            unit += 1
        else:
            unit_details.append(unit_dict)
                        
    #Unit code and Unit title : if condition to extract and store unit_id and title in xml and dictionary.
    if re.search(unitcode_regex, token):
        u_id = re.search(unitcode_regex, token).group(4)
        s_ind = re.search(unitcode_regex, token).end()
        e_ind = token.index("class=\"hbk-archive-only\">")
        title = token[s_ind : e_ind-6]
        if line == 1189:
            xml_output.write("\t</unit>\n")
        else:
            xml_output.write("\t</unit>\n")
        line += 1
        xml_output.write("\t<unit id='" + str(u_id) + "'>\n")
        line += 1
        xml_output.write("\t\t<title>"+ title + "</title>\n")
        line += 1
        unit_dict["@id"] = u_id
        unit_dict["title"] = title
        
    # Synopsis :if condition to extract and store synopsis in xml and dictionary.
    synop_ls = []
    if re.search(synop_regex, token):
        local_index = index
        local_token = textfile_tokens[local_index]
        while not(re.search(outcome_regex,local_token)):
            if re.search(synop_ptag_regex, local_token):
                synop = re.search(synop_ptag_regex,local_token).group(2)
                lr = r"(</p><p>)(.*)(<a) (.*)(</a>)"
                if re.search(lr,synop):
                    r = re.search(lr,synop).group()
                    synop = synop.replace(r,'')
                synop_ls.append(synop)
                xml_output.write("\t\t<synopsis>" + synop + "</synopsis>\n")
                line += 1
            if re.search(synop_litag_regex, local_token):
                synop = re.search(synop_litag_regex,local_token).group(2)
                lr = r"(</p><p>)(.*)(<a) (.*)(</a>)"
                if re.search(lr,synop):
                    r = re.search(lr,synop).group()
                    synop = synop.replace(r,'')
                synop_ls.append(synop)
                xml_output.write("\t\t<synopsis>" + synop + "</synopsis>\n")
                line += 1
            local_index += 1
            local_token = textfile_tokens[local_index]
        unit_dict["synopsis"] = synop_ls

    #Prerequisite : if condition to extract and store prerequisites and corequisites combines in xml and dictionary.
    pre_requistics = {}
    pre_ls = []
    if re.search(prereq_regex, token):
        local_index = index
        local_token = textfile_tokens[local_index]
        xml_output.write("\t\t<pre_requistics>\n")
        line += 1
        while not(re.search(synop_regex, local_token)):
            if re.search(prohibition_regex, token):
                break
            if re.search(pre_regex, local_token):
                prereq = re.search(pre_regex, local_token).group()
                if prereq not in pre_ls:
                    pre_ls.append(prereq)
                    xml_output.write("\t\t\t<pre_requistic>" + prereq + "</pre_requistic>\n")
                    line += 1
            local_index += 1
            local_token = textfile_tokens[local_index]
        xml_output.write("\t\t</pre_requistics>\n")
        line += 1
        pre_requistics["pre_requistic"] = pre_ls
        unit_dict["pre_requistics"] = pre_requistics
        
    #Prohibition : if condition to extract and store Prohibitions in xml and dictionary.
    prohibitions = []
    if re.search(prohibition_regex, token):
        flag = False
        local_index = index
        local_token = textfile_tokens[local_index]
        while not(re.search(synop_regex, local_token)):
            if re.search(pre_regex, local_token):
                    prereq = re.search(pre_regex, local_token).group()
                    if prereq not in prohibitions:
                        prohibitions.append(prereq)
                        xml_output.write("\t\t<prohibitions>" + prereq + "</prohibitions>\n")
                        unit_dict["prohibitions"] = prereq
                        line += 1
                        flag =True
            local_index += 1
            local_token = textfile_tokens[local_index]
        if flag is False:
                prohibitions.append("NA")
                xml_output.write("\t\t<prohibitions>" + "NA" + "</prohibitions>\n")
                unit_dict["prohibitions"] = "NA"
                line += 1

    # Assessment/Requirements : if condition to extract and store requirements in xml and dictionary.
    requirements ={}
    assessment = []
    flag = False
    if re.search(assess_regex, token):
        flag = True
        local_index = index
        local_token = textfile_tokens[local_index]
        xml_output.write("\t\t<requirements>\n")
        line += 1
        while flag:
            if re.search(require_ptag_regex, local_token):
                assess = re.search(require_ptag_regex, local_token).group(2)
                l = len("</p><p>")
                i = 0
                i = assess.find("</p><p>")
                assess = assess[0:i]+assess[i+l:]
                if assess.find("<a href=") is (-1):
                    assessment.append(assess)
                    xml_output.write("\t\t\t<requirement>" + assess + "</requirement>\n")
                    line += 1
            if re.search(require_litag_regex, local_token):
                assess = re.search(require_litag_regex, local_token).group(2)
                assessment.append(assess)
                xml_output.write("\t\t\t<requirement>" + assess + "</requirement>\n")
                line += 1
            elif (re.search(workload_regex, local_token)):
                flag = False
            elif (re.search(last_tag_regex, local_token)):
                flag = False
            local_index += 1
            local_token = textfile_tokens[local_index]
        xml_output.write("\t\t</requirements>\n")
        line += 1
        requirements["requirement"] = assessment
        unit_dict["requirements"] = requirements
        
    # Outcomes : if condition to extract and store Outcomes in xml and dictionary.
    outcomes = {}
    out_ls = []
    if re.search(outcome_regex, token):
        local_index = index
        local_token = textfile_tokens[local_index]
        xml_output.write("\t\t<outcomes>\n")
        line += 1
        while not(re.search(assess_regex,local_token)):
            if re.search(synop_ptag_regex, local_token):
                outcome = re.search(synop_ptag_regex,local_token).group(2)
                lr = r"(</p><p>)(.*)(<a) (.*)(</a>)"
                if re.search(lr,outcome):
                    r = re.search(lr,outcome).group()
                    outcome = outcome.replace(r,'')
                out_ls.append(outcome)
                xml_output.write("\t\t\t<outcome>" + outcome + "</outcome>\n")
                line += 1
            if re.search(synop_litag_regex, local_token):
                outcome = re.search(synop_litag_regex,local_token).group(2)
                lr = r"(</p><p>)(.*)(<a) (.*)(</a>)"
                if re.search(lr,outcome):
                    r = re.search(lr,outcome).group()
                    outcome = outcome.replace(r,'')
                out_ls.append(outcome)
                xml_output.write("\t\t\t<outcome>" + outcome + "</outcome>\n")
                line += 1
            local_index += 1
            local_token = textfile_tokens[local_index]
        xml_output.write("\t\t</outcomes>\n")
        line += 1
        outcomes["outcome"] = out_ls
        unit_dict["outcomes"] = outcomes
        
    #Chief Examiner : if condition to extract and store Chief Examiner in xml and dictionary.
    chief_examiners = {}
    chief_exam = []
    if re.search(ch_ex_regex, token):
        local_index = index
        local_token = textfile_tokens[local_index]
        xml_output.write("\t\t<chief_examiners>\n")
        line += 1
        while not(re.search(coordinator_regex, local_token)):
            if re.search(chex_name_regex,local_token):
                ch_name = re.search(chex_name_regex,local_token).group(2)
                chief_exam.append(ch_name)
                xml_output.write("\t\t\t<chief_examiner>" + ch_name + "</chief_examiner>\n")
                line += 1
            elif re.search("<!-- TBA -->TBA", local_token):
                tba = re.search("<!-- TBA -->TBA", local_token).group()
                chief_exam.append(tba)
                xml_output.write("\t\t\t<chief_examiner>" + "TBA" + "</chief_examiner>\n")
                line += 1
            local_index += 1
            local_token = textfile_tokens[local_index]
        xml_output.write("\t\t</chief_examiners>\n")
        line += 1
        chief_examiners["chief_examiner"] = chief_exam
        unit_dict["chief_examiners"] = chief_examiners
        
# Finally when all the data is extracted of all the units they are stored in list of dictionaries named : unit_details.
# So in last this list is stored in a dictionary named unit_dictionary with key as 'unit' and finally the unit_dictionary is 
# stored in main_dictionary with key as 'units'. This is done so that this format can directly be dumped to json to create the 
# required format.
unit_dictionary["unit"] = unit_details
main_dictionary["units"] = unit_dictionary        

# finally closing the tags.
xml_output.write("\t</unit>\n")
xml_output.write("</units>")
line += 1

# closing the connection with files.
xml_output.close()
text_file.close()

# JSON file : Creating the json file by using the main_dictionary containing all the units.
extracted_json = open("29778271.json",'w')
json.dump(main_dictionary, extracted_json, indent = 2)
extracted_json.close()

 